# HLS Foundation Model Finetuning notebook

This notebook demonstrates the steps to fintune the HLS foundation model (A.K.A Prithvi) which is trained using HLSL30 and HLSS30 datasets. 

Note: Entierty of this notebook is desigend to work well within the AWS sagemaker environment. AWS sagemaker environment access for your account can be found using https://creds-workshop.nasa-impact.net/.

![HLS Training](../images/HLS-training.png)

In [ ]:
# Cell 1: Install required packages
!pip install -r requirements.txt

# Create directories for data, model, and config preparations
!mkdir -p datasets models configs

In [ ]:
# Cell 2: Import Azure ML libraries
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.core.datastore import Datastore
from azureml.train.estimator import Estimator
from azureml.core.runconfig import MpiConfiguration
from azure.storage.blob import BlobServiceClient
from azure.identity import DefaultAzureCredential
import os

In [ ]:
# Cell 3: Initialize Azure ML Workspace
# Method 1: From config file (if running in Azure ML compute)
try:
    ws = Workspace.from_config()
    print(f"Found workspace: {ws.name} in {ws.location}")
except:
    # Method 2: Initialize manually (update with your details)
    ws = Workspace(
        subscription_id='your-subscription-id',
        resource_group='your-resource-group', 
        workspace_name='your-workspace-name'
    )

### Download HLS Burn Scars dataset from Huggingface: https://huggingface.co/datasets/ibm-nasa-geospatial/hls_burn_scars

In [ ]:
# Cell 4: Download HLS Burn Scars dataset from Huggingface
!cd datasets && git clone https://huggingface.co/datasets/ibm-nasa-geospatial/hls_burn_scars && tar -xvzf hls_burn_scars/hls_burn_scars.tar.gz

## Download config and Pre-trained model

The HLS Foundation Model (pre-trained model), and configuration for Burn Scars downstream task are available in Huggingface. We use `huggingface_hub` python package to download the files locally.

In [ ]:
# Cell 5: Define constants - UPDATE THESE VALUES
STORAGE_ACCOUNT_NAME = 'your-storage-account'  # Replace with your storage account
CONTAINER_NAME = 'hls-data'  # Replace with your container name
CONFIG_PATH = './configs'
DATASET_PATH = './datasets' 
MODEL_PATH = './models'
EXPERIMENT_NAME = 'hls-foundation-training'

Note: The configuration file in Huggingface have place holders and won't be directly usable for fine-tuning. Placeholder values need to be updated for individual usecases.

In [ ]:
from huggingface_hub import hf_hub_download

hf_hub_download(
    repo_id="ibm-nasa-geospatial/Prithvi-100M-burn-scar",
    filename="burn_scars_Prithvi_100M.py", 
    local_dir='./configs'
)

In [ ]:
# Cell 7: Update configuration file for Azure ML
# Read the config file and update paths for Azure ML
with open('./configs/burn_scars_Prithvi_100M.py', 'r') as f:
    config_content = f.read()

# Replace paths for Azure ML environment
config_content = config_content.replace(
    "data_root = '<path to data root>'",
    "data_root = '/tmp/data/'"
)
config_content = config_content.replace(
    "pretrained_weights_path = '<path to pretrained weights>'", 
    "pretrained_weights_path = f\"{data_root}/models/prithvi-global-300M.pt\""
)
config_content = config_content.replace(
    "experiment = '<experiment name>'",
    "experiment = 'burn_scars'"
)
config_content = config_content.replace(
    "project_dir = '<project directory name>'",
    "project_dir = 'v1'"
)

# Save updated config
with open('./configs/burn_scars_Prithvi_100M.py', 'w') as f:
    f.write(config_content)

print("✅ Configuration file updated for Azure ML")

In [ ]:
# Cell 8: Upload data to Azure Blob Storage and register as datasets
def upload_to_blob_and_register_dataset(local_path, dataset_name, description):
    """Upload local data to blob storage and register as Azure ML dataset"""
    
    # Get default datastore
    datastore = ws.get_default_datastore()
    
    # Upload data
    print(f"Uploading {local_path} to datastore...")
    datastore.upload(
        src_dir=local_path,
        target_path=dataset_name,
        overwrite=True,
        show_progress=True
    )
    
    # Register as dataset
    dataset = Dataset.File.from_files(
        path=[(datastore, dataset_name)],
        validate=False
    ).register(
        workspace=ws,
        name=dataset_name,
        description=description,
        create_new_version=True
    )
    
    print(f"✅ Registered dataset: {dataset_name}")
    return dataset

# Upload training, validation, and test data
train_dataset = upload_to_blob_and_register_dataset(
    'datasets/training', 
    'hls-training-data',
    'HLS burn scars training data'
)

val_dataset = upload_to_blob_and_register_dataset(
    'datasets/validation',
    'hls-validation-data', 
    'HLS burn scars validation data'
)

test_dataset = upload_to_blob_and_register_dataset(
    'datasets/validation',  # Note: using validation for test as in original
    'hls-test-data',
    'HLS burn scars test data'
)

In [ ]:
# Cell 9: Setup identifier and upload config/model files
identifier = input("Enter your identifier (e.g., your-name): ") or "user"

# Rename config file
config_filename = 'configs/burn_scars_Prithvi_100M.py'
new_config_filename = f"configs/{identifier}-burn_scars_Prithvi_100M.py"
os.rename(config_filename, new_config_filename)

# Upload config to datastore
datastore = ws.get_default_datastore()
datastore.upload_files(
    files=[new_config_filename],
    target_path='configs/',
    overwrite=True
)

# Download and upload pretrained model
print("Downloading pretrained model...")
hf_hub_download(
    repo_id="ibm-nasa-geospatial/Prithvi-100M",
    filename="Prithvi_100M.pt",
    local_dir='./models',
    local_dir_use_symlinks=False
)

# Upload model to datastore
datastore.upload_files(
    files=['./models/Prithvi_100M.pt'],
    target_path='models/',
    overwrite=True
)

print("✅ Config and model files uploaded")

In [ ]:
# Cell 10: Create or get compute target
compute_name = "gpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=compute_name)
    print(f"Found existing compute target: {compute_name}")
except ComputeTargetException:
    print(f"Creating new compute target: {compute_name}")
    
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="Standard_NC6s_v3",  # Equivalent to ml.p3.2xlarge (V100 GPU)
        min_nodes=0,
        max_nodes=1,
        idle_seconds_before_scaledown=300
    )
    
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

In [ ]:
# Cell 11: Create custom environment
from azureml.core import Environment
from azureml.core.environment import DockerBuildContext

# Create environment from your Dockerfile
env = Environment(name="hls-foundation-env")

# Option 1: Use your custom Docker image (if pushed to ACR)
# env.docker.base_image = "your-acr.azurecr.io/azureml-hls:latest"
# env.python.user_managed_dependencies = True

# Option 2: Build from Dockerfile (recommended)
dockerfile_path = "."  # Path to your Dockerfile
env.docker.build_context = DockerBuildContext.from_local_directory(dockerfile_path)
env.docker.dockerfile_path = "Dockerfile"

# Register environment
env.register(workspace=ws)

print("✅ Environment created and registered")

In [ ]:
# Cell 12: Setup training configuration
from azureml.core import ScriptRunConfig

# Environment variables for Azure ML (equivalent to SageMaker environment)
environment_variables = {
    'CONFIG_FILE': f"/tmp/data/configs/{identifier}-burn_scars_Prithvi_100M.py",
    'MODEL_NAME': f"{identifier}-workshop.pth",
    'AZURE_BLOB_URL': f"https://{STORAGE_ACCOUNT_NAME}.blob.core.windows.net/{CONTAINER_NAME}",
    'EVENT_TYPE': 'burn_scars',
    'VERSION': 'v1',
    'DATASET_NAME': 'hls-training-data'  # For dataset-based loading
}

# Create script run configuration
script_config = ScriptRunConfig(
    source_directory='.',  # Directory containing train.py and utils.py
    script='train.py',
    environment=env,
    compute_target=compute_target,
    environment_variables=environment_variables
)

# Add dataset inputs
script_config.run_config.data_references[train_dataset.name] = train_dataset.as_named_input('training').as_mount()
script_config.run_config.data_references[val_dataset.name] = val_dataset.as_named_input('validation').as_mount()  
script_config.run_config.data_references[test_dataset.name] = test_dataset.as_named_input('test').as_mount()

print("✅ Training configuration setup complete")


In [ ]:
# Cell 13: Create and submit experiment
experiment = Experiment(workspace=ws, name=EXPERIMENT_NAME)

print(f"Submitting experiment: {EXPERIMENT_NAME}")
print(f"Compute target: {compute_target.name}")
print(f"Environment: {env.name}")

# Submit the training run
run = experiment.submit(script_config)

print(f"✅ Training job submitted!")
print(f"Run ID: {run.id}")
print(f"Monitor progress at: {run.get_portal_url()}")

In [ ]:
# Cell 14: Monitor training (optional - uncomment to wait for completion)
# run.wait_for_completion(show_output=True)